In [12]:
import time

In [13]:
TOP_ANGLE_VALUE = 1737.0
BOT_ANGLE_VALUE = 1223.0

In [14]:
from angle_sensor import AngleSensor

In [15]:
angle_sensor = AngleSensor('/dev/ttyACM0')
angle_sensor.start()

In [16]:
angle_sensor.get_value()

1530.0

In [16]:
angle_sensor.terminate()

In [17]:
from ros_loop import RosLoop

In [18]:
ros_loop = RosLoop()

In [19]:
from myobrick import MyoBrick

In [20]:
myobrick_flex = MyoBrick(1)
myobrick_extend = MyoBrick(3)

In [21]:
ros_loop.start()

In [22]:
from data_recorder import DataRecorder

In [23]:
data_recorder = DataRecorder(
    20,
    'dataset1.csv',
    myobrick_flex,
    myobrick_extend,
    angle_sensor
)

In [24]:
data_recorder.start()

In [28]:
data_recorder.stop()

In [25]:
def joint_is_within_safe_zone():
    if angle_sensor.get_value() < BOT_ANGLE_VALUE + 50:
        return False
    if angle_sensor.get_value() > TOP_ANGLE_VALUE - 50:
        return False
    
    return True

In [26]:
joint_is_within_safe_zone()

True

In [27]:
direction = 1 # 1 for flex, 0 for extend
while True:
    if not joint_is_within_safe_zone():
        myobrick_extend.set_pwm(0)
        myobrick_flex.set_pwm(0)
        break

    if angle_sensor.get_value() > TOP_ANGLE_VALUE - 100:
        direction = 0

    if angle_sensor.get_value() < BOT_ANGLE_VALUE + 100:
        direction = 1
        

    if direction == 1:
        myobrick_extend.set_pwm(-2)
        myobrick_flex.set_pwm(7)

    if direction == 0:
        myobrick_extend.set_pwm(7)
        myobrick_flex.set_pwm(-2)

    time.sleep(0.1)


KeyboardInterrupt: 

In [1]:
import db1

In [4]:
import pandas as pd

In [29]:
df = pd.read_csv('dataset1.csv')

In [30]:
df = df.set_axis(
    [
        'timestamp',
        'angle',
        'flex_pv_pos_encoder',
        'flex_pv_torque_encoder',
        'flex_pv_current',
        'flex_sp_pwm',
        'extend_pv_pos_encoder',
        'extend_pv_torque_encoder',
        'extend_pv_current',
        'extend_sp_pwm'
    ],
    axis=1, inplace=False)

In [31]:
df

,timestamp,angle,flex_pv_pos_encoder,flex_pv_torque_encoder,flex_pv_current,flex_sp_pwm,extend_pv_pos_encoder,extend_pv_torque_encoder,extend_pv_current,extend_sp_pwm
0,1.670597e+09,1531.0,2.517861,-44082.0,0.0875,0,-0.516989,-104.0,0.1125,0
1,1.670597e+09,1531.0,2.517861,-44082.0,0.0750,0,-0.516989,-104.0,0.0625,0
2,1.670597e+09,1532.0,2.517861,-44082.0,0.1000,0,-0.516989,-104.0,0.0875,0
3,1.670597e+09,1531.0,2.517861,-44082.0,0.1125,0,-0.516989,-104.0,0.0000,0
4,1.670597e+09,1531.0,2.517861,-44082.0,0.0875,0,-0.516989,-104.0,0.0375,0
...,...,...,...,...,...,...,...,...,...,...
821,1.670597e+09,1479.0,2.144064,-43475.0,0.0500,0,-0.193496,-49.0,-0.0125,0
822,1.670597e+09,1480.0,2.144064,-43475.0,0.0625,0,-0.193496,-49.0,0.1500,0
823,1.670597e+09,1479.0,2.144064,-43475.0,0.1375,0,-0.193496,-49.0,0.0125,0
824,1.670597e+09,1479.0,2.144064,-43475.0,0.0250,0,-0.193496,-49.0,0.1375,0


In [32]:
bench_data_item = db1.Item('bench_data')

In [33]:
bench_data_item.val = df